# Bootstrap: Contracts from Existing Files
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/bootstrap_from_file/bootstrap_from_file.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/bootstrap_from_file/bootstrap_from_file.ipynb)



## Business Scenario

Your data engineering team inherits **three years of historical event data**
from a third-party provider — 40+ columns, no documentation, no schema.
You need to onboard these files into your Bronze layer within a sprint.

Writing contracts by hand means:
- Reading column headers, guessing types, documenting meaning
- No quality rules on day one — unknown distributions slip through undetected
- Hours of tedious YAML authoring before a single row can be processed

`infer_contract()` reads the file, infers every column's type, flags
low-cardinality accepted values, detects required fields, and suggests numeric
range bounds — generating a complete contract draft in **one line**.

## Business Value

| Without Bootstrap | With Bootstrap |
|---|---|
| Hours of manual YAML authoring | Contract inferred in seconds |
| Types guessed by hand, often wrong | Polars-native type inference — exact |
| No quality rules on day one | `suggest_rules=True` auto-detects constraints |
| PII discovered late (compliance risk) | `detect_pii=True` flags columns at ingest |
| One-off scripts per file format | Unified API: CSV, Parquet, JSON, NDJSON, Excel |
| Contract disconnected from data | Chain straight into `DataGenerator` |

## Goals

1. Infer a contract from a CSV file and inspect what was generated
2. Auto-suggest quality rules from the data distribution
3. Save the contract to disk and use it with `DataProcessor`
4. Go contract-free: infer directly and generate synthetic data
5. Infer from a JSON file (single-record format)
6. Bootstrap a full landing zone folder via the CLI


In [ ]:
from pathlib import Path
from lakelogic import infer_contract, ContractDraft, DataGenerator, DataProcessor

def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        if base.name == name and base.exists():
            return base
        for sub in [
            base / name,
            base / 'examples' / '06_advanced_workflows' / name,
            base / 'lakelogic' / 'examples' / '06_advanced_workflows' / name,
        ]:
            if sub.exists():
                return sub
    return cwd / name

EXAMPLE_DIR  = resolve_example_dir('bootstrap_from_file')
SYNTH_DIR    = EXAMPLE_DIR.parent / 'synthetic_data_generation'
DATA_DIR     = EXAMPLE_DIR / 'data'
CONTRACT_DIR = EXAMPLE_DIR / 'contracts'
DATA_DIR.mkdir(exist_ok=True)
CONTRACT_DIR.mkdir(exist_ok=True)

# We'll use the orders sample from the synthetic_data_generation example as
# our 'landing zone' data — no need to generate it separately.
ORDERS_CSV     = SYNTH_DIR / 'data' / 'orders_sample.csv'
ORDERS_PARQUET = SYNTH_DIR / 'data' / 'orders_sample.parquet'

print(f'Example dir : {EXAMPLE_DIR}')
print(f'Orders CSV  : {ORDERS_CSV} (exists={ORDERS_CSV.exists()})')


## 1. Infer a Contract from a CSV File

Pass any file path to `infer_contract()` and get back a `ContractDraft`.
No contract YAML needed upfront — the file *is* the spec.

The inferred draft includes:
- A `model.fields` section with every column's name and type
- `required: true` on columns with < 1% nulls
- A `server` block pointing back to the source file
- A `generated_at` timestamp for auditability


In [ ]:
# One line — infer everything from the file
draft = infer_contract(ORDERS_CSV)

print(draft)
print(f'\nFields inferred: {len(draft.fields)}')
print()
for f in draft.fields:
    req = ' (required)' if f.get('required') else ''
    ex  = f'  examples={f["examples"]}' if 'examples' in f else ''
    print(f'  {f["name"]:30s}  {f["type"]}{req}{ex}')


## 2. Auto-Suggest Quality Rules

Set `suggest_rules=True` and the inferrer will scan the data distribution
to generate starter quality rules automatically:

| Pattern detected | Rule generated |
|---|---|
| Column with < 1% nulls | `not_null` SQL rule |
| String column with ≤ 20 distinct values | `accepted_values` rule |
| Numeric column | `range` rule with observed min/max |
| All values distinct across rows | `unique` dataset rule |

These are starter suggestions — review and tune before promoting to Silver.


In [ ]:
draft_with_rules = infer_contract(
    ORDERS_CSV,
    title='Orders Bronze',
    layer='bronze',
    suggest_rules=True,
)

q = draft_with_rules.quality
row_rules     = q.get('row_rules', [])
dataset_rules = q.get('dataset_rules', [])

print(f'Row-level rules   : {len(row_rules)}')
print(f'Dataset-level rules: {len(dataset_rules)}')
print()

for rule in row_rules:
    category = rule.get('category', '')
    print(f'  [{category:15s}]  {rule["name"]:35s}  {rule["sql"][:60]}')

for rule in dataset_rules:
    print(f'  [uniqueness     ]  {rule["name"]:35s}  {rule["sql"]}')


## 3. Inspect and Save the Contract

`ContractDraft.show()` prints the full YAML to stdout — useful for review
before committing to disk. `ContractDraft.save(path)` writes it as a
standard LakeLogic contract file, usable by every other part of the framework.


In [ ]:
# Inspect the full YAML first
draft_with_rules.show()


In [ ]:
# Save to disk — this becomes a first-class LakeLogic contract
contract_path = CONTRACT_DIR / 'bronze_orders_inferred.yaml'
saved = draft_with_rules.save(contract_path)
print(f'Saved: {saved}')
print(f'Size : {saved.stat().st_size:,} bytes')


## 4. Use the Inferred Contract with DataProcessor

The saved contract is now a first-class citizen — plug it into
`DataProcessor` exactly as you would a hand-authored contract.
The rules we auto-generated will enforce quality on every run.


In [ ]:
processor = DataProcessor(contract=contract_path)
result    = processor.run_source(str(ORDERS_PARQUET))

good = result.good
bad  = result.bad
total = (len(good) if good is not None else 0) + (len(bad) if bad is not None else 0)

print(f'Total rows      : {total}')
print(f'Passed          : {len(good) if good is not None else 0}')
print(f'Quarantined     : {len(bad)  if bad  is not None else 0}')


## 5. One-Liner: File → Contract → Synthetic Data

Skip the YAML file entirely. Chain `infer_contract()` → `to_generator()` →
`generate()` to produce realistic synthetic data directly from any file.

Use case: you have a sample, you need 10× more records that look the same,
and you don't want to write a contract first.


In [ ]:
# File → inferred contract → DataGenerator → synthetic data
# No files written to disk — entirely in memory
df_synthetic = (
    infer_contract(ORDERS_CSV)
    .to_generator(seed=42)
    .generate(rows=5_000)
)

print(f'Shape   : {df_synthetic.shape}')
print(f'Columns : {df_synthetic.columns}')
print()
print('Status distribution:')
print(df_synthetic['status'].value_counts().sort('count', descending=True))


## 6. Even Shorter: `DataGenerator.from_file()`

If you only need to generate data (no contract inspection or saving),
`DataGenerator.from_file()` is the minimal API — infer and generate in two lines.


In [ ]:
# No contract, no intermediate — just generate
gen = DataGenerator.from_file(ORDERS_CSV, seed=7)
df  = gen.generate(rows=2_000)

print(f'Shape: {df.shape}')
print('Region distribution:')
print(df['region'].value_counts().sort('count', descending=True))


## 7. Infer from a Single JSON Record

Landing zones often contain **one JSON file per entity** (e.g., one file
per listing, per customer, per event). `infer_contract()` handles
single-record JSON documents automatically — it wraps the object in a list,
infers the schema, and generates a contract.

The generated contract can then be used to produce many synthetic records
that match the shape of the originals.


In [ ]:
import json, polars as pl

# Create a representative single-record JSON in our data dir
# (In a real scenario this would come from your landing zone)
sample_record = {
    'listing_id': 10001,
    'source': 'property',
    'category': 'residential',
    'bathrooms': 2,
    'total_bedrooms': 4,
    'display_address': '14 Grosvenor Road, London',
    'life_cycle_status': 'live',
    'summary_description': 'Spacious Victorian terrace with garden.',
    'creation_date': '2026-01-15T09:30:00Z',
    'pricing': {'price': 875000, 'transaction_type': 'sale'},
    'location': {
        'coordinates': {'latitude': 51.5074, 'longitude': -0.1278},
        'postal_code': 'SW1A 1AA',
        'town_or_city': 'London',
        'country_code': 'GB',
    },
    'derived': {'parking': ['garage'], 'outside_space': ['private_garden']},
}

json_path = DATA_DIR / 'listing_10001.json'
json_path.write_text(json.dumps(sample_record, indent=2), encoding='utf-8')

# Infer from single-record JSON
draft_json = infer_contract(
    json_path,
    title='Zoopla Listing (Bronze)',
    layer='bronze',
    suggest_rules=True,
)

print(draft_json)
print()
for f in draft_json.fields:
    print(f'  {f["name"]:35s}  {f["type"]}')


In [ ]:
# Save the listing contract and generate 500 synthetic listings
listing_contract = CONTRACT_DIR / 'bronze_zoopla_listings_inferred.yaml'
draft_json.save(listing_contract)
print(f'Saved: {listing_contract}')

df_listings = draft_json.to_generator(seed=0).generate(rows=500)
print(f'Generated {len(df_listings)} synthetic listings')
print(df_listings.head(3))


## 8. Infer from Parquet

Same API — just point at a Parquet file. Polars reads the file-embedded
schema for type inference, making Parquet the most accurate source.


In [ ]:
draft_parquet = infer_contract(
    ORDERS_PARQUET,
    title='Orders (from Parquet)',
    suggest_rules=True,
)

print(draft_parquet)
for f in draft_parquet.fields:
    req = ' ✓' if f.get('required') else ''
    print(f'  {f["name"]:30s}  {f["type"]:10s}{req}')


## 9. Infer from an In-Memory DataFrame

Pass a Polars or Pandas DataFrame directly — useful when you've already
loaded and transformed your data and want to capture the resulting schema.


In [ ]:
import polars as pl

df_mem  = pl.read_parquet(ORDERS_PARQUET)

# Infer from the DataFrame — no file path needed
draft_mem = infer_contract(df_mem, title='Orders (in-memory)', suggest_rules=False)

print(draft_mem)
print(f'Fields: {len(draft_mem.fields)}')

# Chain into generator immediately
df_mem_synth = draft_mem.to_generator(seed=1).generate(rows=200)
print(f'Generated: {df_mem_synth.shape}')


## 10. Bootstrap a Full Landing Zone — CLI

For **bulk onboarding** — scan an entire folder, generate one contract per
entity, and write a registry that makes every contract immediately runnable
by `lakelogic-driver`.

```bash
# Scan a folder, infer contracts, write a registry
lakelogic bootstrap \
  --landing data/landing \
  --output-dir contracts/bronze \
  --registry contracts/bronze/_registry.yaml \
  --format csv \
  --pattern '*.csv' \
  --suggest-rules

# Sync mode: pick up new files without overwriting existing contracts
lakelogic bootstrap \
  --landing data/landing \
  --output-dir contracts/bronze \
  --registry contracts/bronze/_registry.yaml \
  --sync

# Add new columns to existing contracts without regenerating them
lakelogic bootstrap \
  --landing data/landing \
  --output-dir contracts/bronze \
  --registry contracts/bronze/_registry.yaml \
  --sync --sync-update-schema

# PII detection (requires: pip install lakelogic[profiling])
lakelogic bootstrap \
  --landing data/landing \
  --output-dir contracts/bronze \
  --registry contracts/bronze/_registry.yaml \
  --detect-pii
```

The resulting `_registry.yaml` can drive the full pipeline:
```bash
lakelogic-driver --registry contracts/bronze/_registry.yaml --layers bronze
```


## 11. ContractDraft API Reference

`infer_contract()` always returns a `ContractDraft`. Every method on it:

| Method / Property | Description |
|---|---|
| `.fields` | List of `{name, type, required?, examples?}` dicts |
| `.quality` | Suggested quality rules dict |
| `.show()` | Print YAML to stdout |
| `.to_yaml()` | Return YAML as a string |
| `.to_dict()` | Return contract as a plain Python dict |
| `.save(path)` | Write YAML to disk |
| `.to_generator(seed)` | Return a `DataGenerator` backed by this contract |


In [ ]:
# ContractDraft is serialisable in every direction
draft_with_rules = infer_contract(ORDERS_CSV, suggest_rules=True)

# As a dict (e.g. for REST API payloads or further manipulation)
d = draft_with_rules.to_dict()
print('Dict keys:', list(d.keys()))

# As a YAML string (e.g. to store in a database or display in a UI)
yml = draft_with_rules.to_yaml()
print(f'\nYAML length: {len(yml)} chars')
print(yml[:300], '...')


---

## Summary

| Scenario | API |
|---|---|
| Infer from a file (inspect / save) | `infer_contract(file)` → `ContractDraft` |
| Infer with quality rules | `infer_contract(file, suggest_rules=True)` |
| Infer with PII detection | `infer_contract(file, detect_pii=True)` |
| Save contract to disk | `draft.save('contracts/bronze_entity.yaml')` |
| Print contract YAML | `draft.show()` |
| Get contract as string | `draft.to_yaml()` |
| Chain to generator (no file) | `infer_contract(file).to_generator().generate(rows=N)` |
| Quick generate, no contract | `DataGenerator.from_file(file).generate(rows=N)` |
| Bulk landing zone bootstrap | `lakelogic bootstrap --landing ...` |
| Sync (add new entities only) | `lakelogic bootstrap ... --sync` |

### Decision guide

```
Have a contract already?                   ── Yes ──► DataGenerator(contract).generate()
        │
        No
        │
        ├── Need to save / review the contract? ── Yes ──► infer_contract().save()
        │
        └── Just need synthetic data quickly?  ── Yes ──► DataGenerator.from_file().generate()
```
